# Ejercicio AirBnB

En este cuaderno vamos a trabajar con un dataset de AirBnB de la ciudad de Oporto. Se puede encontrar más información sobre el dataset y otras implementaciones en [Porto](https://github.com/Vasallo94/Porto). 

El dataset contiene información sobre las características de las viviendas, su localización, el precio, el número de comentarios, etc.

El objetivo de este ejercicio es poder analizar los comentarios de los usuarios mediante diferentes LLMs (GPT y LLAMA) y poder extraer información relevante de los mismos para su análisis posterior.

## Primera parte

Vamos a utilizar el archivo listings1_cleaned.csv que contiene información sobre las viviendas de AirBnB en Oporto.

In [1]:

import pandas as pd
import sys
sys.path.append('../utils/')
from funciones import preguntar_llama, preguntar_gpt
import json

In [2]:
data = pd.read_csv("listings1_cleaned.csv")
data

,listing_id,listing_url,picture_url,name,description,host_id,host_name,host_since,host_response_rate,host_acceptance_rate,...,review_scores_rating,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,reviews_per_month,availability_365,has_availability,last_review,geographical_group
0,2.905900e+04,https://www.airbnb.com/rooms/29059,https://a0.muscache.com/pictures/736399/fa6c31...,Lovely studio Quartier Latin,CITQ 267153<br />Lovely studio with 1 closed r...,125031.0,Maryline,2010-05-14,100.000000,98.000000,...,4.670000,4.620000,4.810000,4.770000,4.820000,2.69000,302.0,t,2024-03-16,Central
1,2.906100e+04,https://www.airbnb.com/rooms/29061,https://a0.muscache.com/pictures/9e59d417-4b6a...,Maison historique - Quartier Latin,Lovely historic house with plenty of period ch...,125031.0,Maryline,2010-05-14,100.000000,98.000000,...,4.730000,4.660000,4.880000,4.810000,4.870000,0.88000,348.0,t,2024-02-19,Central
2,3.630100e+04,https://www.airbnb.com/rooms/36301,https://a0.muscache.com/pictures/26c20544-475f...,Romantic & peaceful Plateau loft,"Enjoy the best of Montreal in this romantic, ...",381468.0,Sylvie,2011-02-07,94.000000,80.000000,...,4.860000,4.860000,4.920000,4.900000,4.880000,0.47000,81.0,t,2024-01-07,Central
3,3.811800e+04,https://www.airbnb.com/rooms/38118,https://a0.muscache.com/pictures/213997/763ec1...,Beautiful room with a balcony in front of a parc,Nearest metro Papineau.,163569.0,M.,2010-07-11,78.000000,0.000000,...,4.500000,4.250000,4.810000,4.810000,4.630000,0.10000,299.0,t,2022-08-29,Central
4,5.047900e+04,https://www.airbnb.com/rooms/50479,https://a0.muscache.com/pictures/miso/Hosting-...,L'Arcade Douce,The appartement is sunny and ideally situated ...,231694.0,Noemie,2010-09-11,100.000000,100.000000,...,4.950000,4.940000,4.970000,4.980000,4.840000,1.60000,0.0,t,2024-03-18,South
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8135,1.116753e+18,https://www.airbnb.com/rooms/1116753448972266015,https://a0.muscache.com/pictures/miso/Hosting-...,"Logement ,1 chambre 04 personnes",If you are looking for a beautiful cozy and we...,429774011.0,Ammar,2021-10-31,96.922347,89.203748,...,4.715698,4.692737,4.814562,4.812302,4.768682,1.70759,257.0,t,NaN,North
8136,1.116820e+18,https://www.airbnb.com/rooms/1116820020797670135,https://a0.muscache.com/pictures/miso/Hosting-...,Superbe maison à Montréal,Beautiful entire place in the heart of Montreal,126764590.0,Samuel,2017-04-20,96.922347,89.203748,...,4.715698,4.692737,4.814562,4.812302,4.768682,1.70759,269.0,t,NaN,East
8137,1.117028e+18,https://www.airbnb.com/rooms/1117028063118176710,https://a0.muscache.com/pictures/miso/Hosting-...,Luxurious 2BR - Downtown Montreal,Feel at home at these brand new construction C...,159008278.0,Melissa,2017-11-16,100.000000,100.000000,...,4.715698,4.692737,4.814562,4.812302,4.768682,1.70759,156.0,t,NaN,Central
8138,1.117273e+18,https://www.airbnb.com/rooms/1117273423317641658,https://a0.muscache.com/pictures/miso/Hosting-...,Magnifique appart Plateau,"Fully renovated condo style apartment, beautif...",214303569.0,Jean-Georges,2018-09-08,100.000000,93.000000,...,4.715698,4.692737,4.814562,4.812302,4.768682,1.70759,192.0,t,NaN,Central


In [3]:
data = data[["listing_id", "name", "description"]]
data["text"] = data["name"].fillna("") + "\n" + data["description"].fillna("")
data = data.sample(3, random_state=10)

llama_responses = {}

prompt_template = """
Vas a analizar los comentarios de unos alojamientos turísticos. 
Se te va a proporcionar una serie de textos que has de analizar y clasificar en un json con la siguiente estructura:
```json
{
    "Name": str,
    "Location": str,
    "MainCharacteristic": str,
    "Type": str,
    
}
```
Donde:
- Name: Nombre del alojamiento.
- Location: Ubicación del alojamiento.
- MainCharacteristic: Característica principal del alojamiento.
- Type: Tipo de alojamiento.
Analiza el siguiente texto y clasifícalo en el json correspondiente: 
```
{{text}}
```
"""
for index, row in data.iterrows():
    text = str(row["text"])
    listing_id = row["listing_id"]

    prompt = prompt_template.replace("{{text}}", text)
    # Cambiar a preguntar_gpt para utilizar OpenAI
    llama_response = preguntar_gpt(
        "Eres un asistente de IA cuyo trabajo es responder a lo que te está pidiendo el usuario de la forma más fiel al prompt sin preámbulos explicativos ni resúmenes finales.",
        prompt,
    )
    if llama_response is not None:
        llama_responses[listing_id] = llama_response
    else:
        print(f"No se pudo obtener una respuesta para el listing_id: {listing_id}")

data["gpt_response"] = data["listing_id"].map(llama_responses)
data

/var/folders/l7/2cl9nrjs74bc3y72c8n3j2k80000gp/T/ipykernel_38233/2643287345.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["text"] = data["name"].fillna("") + "\n" + data["description"].fillna("")


,listing_id,name,description,text,gpt_response
1750,3.575908e+07,Bright flat in Montreal - Appartement lumineux,Bright 2 bedroom flat in a friendly and authen...,Bright flat in Montreal - Appartement lumineux...,"```json\n{\n ""Name"": ""Bright flat in Montre..."
66,6.375330e+05,Charming Downtown Montreal,Charm & standing for your stay in Montreal<br ...,Charming Downtown Montreal\nCharm & standing f...,"{\n ""Name"": ""Charming Downtown Montreal"",\n..."
4598,7.822780e+17,Stunning 2BR - Old Montreal 2A,Welcome to Hotel Le Moyne; Experience the allu...,Stunning 2BR - Old Montreal 2A\nWelcome to Hot...,"```json\n{\n ""Name"": ""Hotel Le Moyne"",\n ..."


In [4]:
data.to_csv('Airbnb_gpt_15responses.csv', index=False)

In [5]:
data = pd.read_csv('Airbnb_gpt_15responses.csv')

In [6]:
data

,listing_id,name,description,text,gpt_response
0,3.575908e+07,Bright flat in Montreal - Appartement lumineux,Bright 2 bedroom flat in a friendly and authen...,Bright flat in Montreal - Appartement lumineux...,"```json\n{\n ""Name"": ""Bright flat in Montre..."
1,6.375330e+05,Charming Downtown Montreal,Charm & standing for your stay in Montreal<br ...,Charming Downtown Montreal\nCharm & standing f...,"{\n ""Name"": ""Charming Downtown Montreal"",\n..."
2,7.822780e+17,Stunning 2BR - Old Montreal 2A,Welcome to Hotel Le Moyne; Experience the allu...,Stunning 2BR - Old Montreal 2A\nWelcome to Hot...,"```json\n{\n ""Name"": ""Hotel Le Moyne"",\n ..."


In [7]:
def process_json_response(json_str):
    # Verificar si json_str es NaN
    if json_str != json_str:  # Esta es una forma de verificar NaN en Python
        return {"error": "Input is NaN, cannot process."}

    # Limpiar la entrada si está envuelta en bloques de código markdown
    if json_str.startswith("```json\n") and json_str.endswith("```"):
        json_str = json_str[7:-3]  # Eliminar los marcadores de bloque de código

    try:
        # Intentar cargar el JSON
        json_data = json.loads(json_str)
        return json_data
    except json.JSONDecodeError:
        return {}


# Aplicar la función a la columna 'gpt_response'
data["processed_response"] = data["gpt_response"].apply(
    process_json_response
)  # Cambiar a 'gpt_response' si es necesario

# Encontrar el primer elemento no vacío (que no sea {}) para determinar el orden de las claves
first_valid_response = next((item for item in data["processed_response"] if item), None)

if first_valid_response:
    # Obtener todas las claves únicas de los JSONs en el orden del primer elemento válido
    ordered_keys = list(first_valid_response.keys())

    # Crear nuevas columnas para cada clave, manteniendo el orden
    for key in ordered_keys:
        data[key] = data["processed_response"].apply(lambda x: x.get(key, None))

# Eliminar la columna temporal 'processed_response'
data = data.drop("processed_response", axis=1)

data

,listing_id,name,description,text,gpt_response,Name,Location,MainCharacteristic,Type
0,3.575908e+07,Bright flat in Montreal - Appartement lumineux,Bright 2 bedroom flat in a friendly and authen...,Bright flat in Montreal - Appartement lumineux...,"```json\n{\n ""Name"": ""Bright flat in Montre...",Bright flat in Montreal - Appartement lumineux,Montreal,Bright 2 bedroom flat in a friendly and authen...,Apartment
1,6.375330e+05,Charming Downtown Montreal,Charm & standing for your stay in Montreal<br ...,Charming Downtown Montreal\nCharm & standing f...,"{\n ""Name"": ""Charming Downtown Montreal"",\n...",Charming Downtown Montreal,Montreal,Completely renovated apartments in a private m...,Tourist residence
2,7.822780e+17,Stunning 2BR - Old Montreal 2A,Welcome to Hotel Le Moyne; Experience the allu...,Stunning 2BR - Old Montreal 2A\nWelcome to Hot...,"```json\n{\n ""Name"": ""Hotel Le Moyne"",\n ...",Hotel Le Moyne,Old Montreal,Breathtaking views of historical charm,2-bedroom apartment


In [8]:
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px
import ast
import json


def string_to_list(x):
    if isinstance(x, str):
        try:
            return ast.literal_eval(x)
        except:
            return x
    return x


def count_list_elements(series):
    counter = {}
    for item in series:
        if isinstance(item, list):
            for element in item:
                if isinstance(element, dict):
                    element = json.dumps(element, sort_keys=True)
                counter[element] = counter.get(element, 0) + 1
        else:
            if isinstance(item, dict):
                item = json.dumps(item, sort_keys=True)
            counter[item] = counter.get(item, 0) + 1
    return pd.Series(counter)


def create_plot(df, column):
    series = df[column].apply(string_to_list)
    if series.dtype == "O":
        if series.apply(lambda x: isinstance(x, list)).any():
            series = series.apply(lambda x: x if isinstance(x, list) else [x])
            flat_list = [item for sublist in series for item in sublist]
            counts = pd.Series(flat_list).value_counts()
        else:
            counts = series.value_counts()
        fig = px.bar(
            counts,
            x=counts.index,
            y=counts.values,
            labels={"y": "count", "index": column},
        )
    elif series.dtype in ["int64", "float64"]:
        fig = px.histogram(series, x=column)
    else:
        print(f"Tipo de datos no soportado para la columna {column}")
        return None
    return fig


new_columns = [
    col
    for col in data.columns
    if col
    not in [
        "listing_id",
        "name",
        "description",
        "text",
        "gpt_response",
        "llama_response",
    ]
]

# Ajustes para mejorar la disposición
altura_base_por_subplot = 500  # Aumentado aún más para dar espacio adicional
altura_total = altura_base_por_subplot * len(new_columns)
espaciado_vertical = 0.25  # Aumentado para mayor separación entre subplots

fig = make_subplots(
    rows=len(new_columns),
    cols=1,
    subplot_titles=new_columns,
    vertical_spacing=espaciado_vertical,
)

for i, column in enumerate(new_columns, start=1):
    subplot = create_plot(data, column)
    if subplot:
        for trace in subplot.data:
            fig.add_trace(trace, row=i, col=1)

        # Actualizar el diseño de cada subplot individualmente
        fig.update_xaxes(
            title_text=column, row=i, col=1, tickangle=45
        )  # Rotar etiquetas 45 grados
        fig.update_yaxes(title_text="Count", row=i, col=1)

        # Ajuste especial para columnas con etiquetas largas (como "3goodthings")
        if (
            "3goodthings" in column or len(column) > 15
        ):  # Ajusta este valor según sea necesario
            fig.update_xaxes(
                tickmode="array", tickvals=[], ticktext=[], row=i, col=1
            )  # Ocultar etiquetas del eje X
            fig.update_annotations(
                text=column, row=i, col=1, x=0.5, y=-0.15, xref="paper", yref="paper"
            )  # Mover título del eje X hacia abajo

# Actualizar el diseño global
fig.update_layout(
    height=altura_total,
    title_text="Visualizaciones de las nuevas columnas",
    showlegend=False,
    title_x=0.5,  # Centrar el título principal
    margin=dict(t=50, b=100, l=50, r=50),  # Aumentar el margen inferior
)

# Ajustar la posición de los títulos de los subplots
for i in fig["layout"]["annotations"]:
    i["y"] = i["y"] + 0.05  # Mover los títulos de los subplots más hacia arriba

# Mostrar la figura
fig.show()

## Parte 2

Realiza un análisis de los comentarios de los apartamentos con un modelo de lenguaje natural (GPT o LLAMA) y extrae información relevante de los mismos.

Por ejemplo, puedes analizar los comentarios y extraer información sobre la limpieza, la ubicación, la relación calidad-precio, el sentimiento del comentario, etc.

In [9]:
comentarios = pd.read_csv("Airbnb_reviews_5000.csv")
comentarios

,name,host_id,host_name,date,reviewer_id,reviewer_name,comments,language
0,LUXURY apartment t3 oporto antas,37249350.0,Joaquim,2018-01-30,156292241.0,Silvia,Acomodação espaçosa e bonita. O Joaquim e sua ...,pt
1,Aida's Haven | Room&PrivateBath | St. Catarina,38365612.0,Alexandra,2019-12-15,283158910.0,Tierry Dayan,"A anfitriã Alexandra foi muito simpática, pre...",pt
2,Maritime Inspiration - One Bedroom Beach Apart...,147469727.0,Susana,2018-08-12,173756309.0,Julio,"Apartamento muy bien comunicado, aunque nos es...",es
3,Central charming Top floor - nice views,26222276.0,A.Maria,2016-05-28,11705876.0,Rafael,"Adosinda was kind, showed the apartment, set u...",en
4,Ribeira Oporto Apartment II (Renewed 2021),35057317.0,João,2016-06-02,38269559.0,Anais,"Un appartement au coeur de porto, très grand a...",fr
...,...,...,...,...,...,...,...,...
4995,Marquês's House - Estúdio ao Marquês,93991335.0,Conceição,2017-09-10,13710198.0,Sandra,Merci pour le chaleureux accueil et tous les b...,fr
4996,M1.4 | Surf Beach Matosinhos | Porto,39551356.0,Porto Je T'Aime,2018-08-04,204048434.0,Louis,Toutes les qualités recherchées dans ce type d...,fr
4997,Kitchnette Studio in Porto's Downtown II,1651078.0,Lurdes,2022-06-19,68564036.0,Angie,"Its a lovely little studio, a green oasis, rig...",en
4998,MyLoft-Charming Apartment!!! - Metro Bolhão,34348897.0,Carla,2022-06-23,283375850.0,Angelika,Carla ist wirklich eine sehr tolle Gastgeberin...,de
